In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans
import plotly.express as px
import umap
import torch
from torch import nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler

2026-01-25 02:49:51.669739: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769309391.947157      17 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769309392.028379      17 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769309392.642518      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769309392.642560      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769309392.642563      17 computation_placer.cc:177] computation placer alr

In [2]:
# Загрузка
OrdersDataset = pd.read_csv(
    "/kaggle/input/restaurant-transactions/restaurant_orders.csv"
)

# Приведение дат к datetime
OrdersDataset["open_time"] = pd.to_datetime(
    OrdersDataset["open_time"], format="%Y-%m-%d %H:%M:%S"
)
OrdersDataset["check_date"] = pd.to_datetime(
    OrdersDataset["check_date"], format="%Y-%m-%d %H:%M:%S"
)

# Признак оплаты: наличные / безналичные
OrdersDataset["is_cash"] = OrdersDataset["payment_type"].apply(lambda x: 1 if x == "Наличный" else 0)

# Группировка заказов (один заказ = одна строка)
OrdersDataset = (
    OrdersDataset
    .groupby("order_id", as_index=False)
    .agg(
        # ----- данные заказа -----
        open_time=("open_time", "first"),
        total_cost=("total_cost", "first"),
        status_name=("status_name", "first"),

        # ----- посетитель -----
        visitor_passport=("visitor_passport", "first"),
        bank_card=("bank_card", "first"),
        visitor_login=("visitor_login", "first"),

        # ----- стол -----
        table_id=("table_id", "first"),
        table_label=("table_label", "first"),
        seat_count=("seat_count", "first"),
        table_zone=("table_zone", "first"),

        # ----- оплата -----
        check_id=("check_id", "first"),
        check_date=("check_date", "first"),
        payment_type=("payment_type", "first"),
        amount_paid=("amount_paid", "first"),
        change_amount=("change_amount", "first"),

        # ----- блюда (склеиваем списки) -----
        menu_name=("menu_name", lambda x: "; ".join(x)),
        quantity=("quantity", "sum"),
        is_cash=("is_cash", "first")
    )
)

# Проверка
OrdersDataset.head()


,order_id,open_time,total_cost,status_name,visitor_passport,bank_card,visitor_login,table_id,table_label,seat_count,table_zone,check_id,check_date,payment_type,amount_paid,change_amount,menu_name,quantity,is_cash
0,1,2023-09-01 14:00:24,2890.0,Выдан,4510665764,4825773177881752,IvanovII,3,ОБ3,2,Общая зона,1,2023-09-01 18:56:54,Наличный,3500.0,32.0,Филе порося; Суп мечты; Картофель по-своему,5,1
1,2,2023-09-01 16:17:37,1070.0,Ожидается,4515009426,313477425677,PavlovEG,1,ОБ1,4,Общая зона,2,2023-09-03 15:21:47,Безналичный,5484.0,0.0,Картофель по-своему; Филе порося; Суп мечты,5,0
2,3,2023-09-03 12:10:41,4570.0,Выдан,4678239712,565211479921,PetrovAA,8,Ч1,4,Частная зона,3,2023-09-04 20:02:52,Наличный,3000.0,300.0,Гарнир овощной; Филе порося; Гарнир овощной; М...,5,1
3,4,2025-10-12 17:07:39,1000.0,Ожидается,7826241549,2292242442374808,haritonovrostislav,14,T1,5,Барная зона,4,2025-10-12 17:07:39,Наличный,1480.0,480.0,Пицца Маргарита,1,1
4,5,2025-11-23 06:07:39,5250.0,Выдан,5692998094,2472930011952614,antonovermil,41,T51,1,Некурящая зона,5,2025-11-23 06:07:39,Наличный,6140.0,890.0,Рататуй; Севиче; Как бы здоровое питание; Суп ...,6,1


In [3]:
def assign_class_enhanced(row):
    """
    0 - Эконом
    1 - Семейный/Групповой
    2 - Премиум
    3 - Бизнес
    """
    seat = row["seat_count"]
    total = row["total_cost"]
    zone = row["table_zone"]
    dishes = row["quantity"]
    is_cash = row["is_cash"]
    paid = row["amount_paid"]
    change = row["change_amount"]

    avg_price = total / dishes if dishes else 0.0

    EPS = 1e-2
    paid_exact = abs(paid - total) < EPS
    has_change = change > EPS

    # Пороговые константы
    PREMIUM_TOTAL = 16000
    PREMIUM_AVG = 1800
    PREMIUM_TOTAL_LOWER = 12000
    PREMIUM_AVG_LOWER = 1200

    BUSINESS_MIN = 7000
    BUSINESS_MAX = 16000

    FAMILY_DISHES = 6
    FAMILY_SEATS = 6
    FAMILY_DISHES_ALT = 4
    FAMILY_SEATS_ALT = 4
    FAMILY_ZONE = {"Семейная", "Детская"}

    ECON_TOTAL_MAX = 5000
    ECON_SEATS_MAX = 3
    ECON_DISHES_MAX = 2
    ECON_AVG_MAX = 1000

    VIP_ZONES = {"VIP", "Частная зона"}
    BUSINESS_ZONES = {"Бизнес-зал", "Зал для мероприятий"}

    # Список правил: (условие, класс)
    rules = [
        # Премиум
        (
            (
                total >= PREMIUM_TOTAL
                or avg_price >= PREMIUM_AVG
                or zone in VIP_ZONES
                or (is_cash and paid >= total * 1.3)
                or (total >= PREMIUM_TOTAL_LOWER and avg_price >= PREMIUM_AVG_LOWER and paid_exact)
            ),
            2,
        ),
        # Бизнес
        (
            (
                (BUSINESS_MIN <= total < BUSINESS_MAX)
                and paid_exact
                and not has_change
                and (zone in BUSINESS_ZONES or dishes >= 3)
            ),
            3,
        ),
        # Семейные / групповые
        (
            (
                dishes >= FAMILY_DISHES
                or seat >= FAMILY_SEATS
                or (dishes >= FAMILY_DISHES_ALT and seat >= FAMILY_SEATS_ALT)
                or zone in FAMILY_ZONE
                or (dishes >= 3 and total >= 6000)
            ),
            1,
        ),
        # Эконом
        (
            (
                total <= ECON_TOTAL_MAX
                and seat <= ECON_SEATS_MAX
                and dishes <= ECON_DISHES_MAX
                and avg_price <= ECON_AVG_MAX
                and has_change
            ),
            0,
        ),
        # По умолчанию — Эконом
        (True, 0),
    ]

    return next(cls for cond, cls in rules if cond)

# Применяем ко всему датасету
OrdersDataset["client_class_id"] = OrdersDataset.apply(assign_class_enhanced, axis=1)

# Логика присвоения классов клиентов

Функция `assign_class_enhanced` присваивает каждому заказу один из четырех классов:

| Класс | Описание             |
| ----- | -------------------- |
| 0     | Эконом               |
| 1     | Семейный / Групповой |
| 2     | Премиум              |
| 3     | Бизнес               |


## Принцип работы

Для каждого заказа функция проверяет **набор условий**, которые идут по **приоритету** (сначала премиум, потом бизнес, семейные, эконом). Первый выполненный блок определяет класс заказа.


### 1. Премиум (2)

Класс присваивается, если выполняется **любое** из условий:

* `total >= 16 000` — общая сумма заказа очень большая
* `avg_price >= 1 800` — средняя цена блюда высокая
* `zone` в VIP-зоне (`VIP`, `Частная зона`)
* Оплата наличными с переплатой: `is_cash` и `paid >= total * 1.3`
* Логическая гарантия: `total >= 12 000` и `avg_price >= 1 200` и `paid_exact`

> Эти условия обеспечивают, что заказ с высокими суммами и премиальными характеристиками попадет в премиум.


### 2. Бизнес (3)

Присваивается, если одновременно выполняются условия:

* `7 000 <= total < 16 000` — средний чек бизнес-уровня
* `paid_exact` — оплата ровно по счету
* `not has_change` — без сдачи
* И `zone` в бизнес-зале (`Бизнес-зал`, `Зал для мероприятий`) **или** `dishes >= 3` — заказ для нескольких человек

> Это гарантирует, что бизнес-класс выделяется по сумме и организации оплаты, а также по месту проведения.


### 3. Семейный / Групповой (1)

Присваивается, если выполняется **любое** из условий:

* `dishes >= 6` — много блюд, крупная группа
* `seat >= 6` — много мест за столом
* `(dishes >= 4 and seat >= 4)` — средняя группа
* `zone` в `{"Семейная", "Детская"}` — семейная зона
* `dishes >= 3 and total >= 6 000` — небольшая группа с дорогим заказом

> Таким образом, учитывается как количество людей, так и общее количество блюд.


### 4. Эконом (0)

Присваивается, если **все** условия выполняются:

* `total <= 5 000` — маленький чек
* `seat <= 3` — небольшая группа
* `dishes <= 2` — мало блюд
* `avg_price <= 1 000` — недорогие блюда
* `has_change` — есть сдача (оплата наличными)

> Эконом-класс — это небольшие заказы с минимальными затратами.


### 5. По умолчанию

Если ни одно из вышеуказанных условий не сработало, заказ присваивается **эконом-класс (0)**.


In [4]:
# Числовые признаки для кластеризации
numeric_features = [
    "total_cost", "seat_count", "quantity",
    "amount_paid", "change_amount", "is_cash"
]

X = OrdersDataset[numeric_features]

# Стандартизация признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Кластеризация MiniBatchKMeans
n_clusters = 4
model = MiniBatchKMeans(
    n_clusters=n_clusters,
    max_iter=100,
    batch_size=32,
    random_state=42
)
clusters = model.fit_predict(X_scaled)

# -----------------------------
# UMAP для 2D-визуализации
# -----------------------------


reducer = umap.UMAP(n_neighbors=15, min_dist=0.1)
embedding = reducer.fit_transform(X_scaled)
OrdersDataset["UMAP_1"] = embedding[:, 0]
OrdersDataset["UMAP_2"] = embedding[:, 1]

# -----------------------------
# Визуализация кластеров
# -----------------------------
fig = px.scatter(
    OrdersDataset,
    x="UMAP_1",
    y="UMAP_2",
    color="client_class_id",
    hover_data=[
        "total_cost", "seat_count", "quantity", "payment_type", "client_class_id"
    ],
    title="UMAP визуализация кластеров заказов",
    color_discrete_sequence=px.colors.qualitative.Prism
)
fig.update_traces(marker=dict(size=12))
fig.show()


In [5]:
# Подготовка данных
class_counts = OrdersDataset["client_class_id"].value_counts().sort_index().reset_index()
class_counts.columns = ["client_class_id", "count"]
class_names = {0: "Эконом", 1: "Семейные/групповые", 2: "Премиум", 3: "Бизнес"}
class_counts["client_class"] = class_counts["client_class_id"].map(class_names)

# Визуализация через Plotly
fig = px.bar(
    class_counts,
    x="client_class",
    y="count",
    text="count",
    title="Распределение классов клиентов",
    color="client_class",
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.update_traces(textposition='outside')
fig.update_layout(showlegend=False)
fig.show()


# Почему мы использовали `WeightedRandomSampler`  

В нашем датасете классы клиентов сильно **несбалансированы**:  

* Эконом — мало заказов  
* Семейные/групповые — много заказов  
* Премиум — среднее количество  
* Бизнес — меньше среднего  

Если обучать модель напрямую на таком датасете, она будет **предпочитать классы с большим количеством примеров**, что приведет к плохому качеству классификации для малых классов (низкий recall и F1-score для Эконом и Бизнес).  

## Что делает `WeightedRandomSampler`

* Каждому примеру в тренировочном датасете присваивается **вес**, обратный частоте его класса: чем меньше примеров класса — тем выше вес.  
* Во время обучения DataLoader выбирает примеры **с учетом этих весов**, что позволяет **искусственно сбалансировать обучение** без генерации новых данных (как в SMOTE).  
* Таким образом, модель видит примерно одинаковое количество примеров каждого класса в каждой эпохе, и обучение становится более **справедливым для всех классов**.  

## Преимущества

1. Не изменяет исходный датасет (нет синтетических данных).  
2. Легко интегрируется с PyTorch DataLoader.  
3. Подходит для **классов с сильно различающейся частотой**, как у нас.  

**Вывод:** WeightedRandomSampler позволяет модели обучаться **на всех классах равномерно**, улучшая F1-score для редких классов без потери качества для часто встречающихся.


In [6]:
# -----------------------------
# 1. Признаки и таргет
# -----------------------------
features = [
    "total_cost",
    "seat_count",
    "quantity",
    "amount_paid",
    "change_amount",
    "is_cash"
]

X = OrdersDataset[features].fillna(0)
y = OrdersDataset["client_class_id"].values

# -----------------------------
# 2. Стандартизация
# -----------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# -----------------------------
# 3. Train / Test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# -----------------------------
# 4. WeightedRandomSampler (борьба с дисбалансом)
# -----------------------------
# Вычисляем веса классов
class_sample_counts = np.bincount(y_train)
weights = 1.0 / class_sample_counts
samples_weight = weights[y_train]
sampler = WeightedRandomSampler(
    weights=samples_weight,
    num_samples=len(samples_weight),
    replacement=True
)

# -----------------------------
# 5. В torch
# -----------------------------
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

X_all_tensor = torch.tensor(X_scaled, dtype=torch.float32)

train_loader = DataLoader(
    TensorDataset(X_train_tensor, y_train_tensor),
    batch_size=16,
    sampler=sampler
)

test_loader = DataLoader(
    TensorDataset(X_test_tensor, y_test_tensor),
    batch_size=16,
    shuffle=False
)

# -----------------------------
# 6. Модель
# -----------------------------
class ClassifierNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=32, output_dim=4):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

model = ClassifierNN(input_dim=X_train_tensor.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# -----------------------------
# 7. Обучение
# -----------------------------
epochs = 10
for epoch in range(epochs):
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()

    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            out = model(xb)
            preds.extend(torch.argmax(out, dim=1).tolist())
            labels.extend(yb.tolist())

    f1 = f1_score(labels, preds, average="weighted")
    print(f"Epoch {epoch+1}/{epochs} | F1-score: {f1:.4f}")

# -----------------------------
# 8. Предсказания на всем датасете
# -----------------------------
model.eval()
with torch.no_grad():
    preds_all = torch.argmax(model(X_all_tensor), dim=1).numpy()

OrdersDataset["client_class_pred"] = preds_all

# Проверка
OrdersDataset[["client_class_id", "client_class_pred"]].head(10)


Epoch 1/10 | F1-score: 0.6218
Epoch 2/10 | F1-score: 0.6775
Epoch 3/10 | F1-score: 0.7643
Epoch 4/10 | F1-score: 0.8037
Epoch 5/10 | F1-score: 0.8418
Epoch 6/10 | F1-score: 0.8242
Epoch 7/10 | F1-score: 0.8434
Epoch 8/10 | F1-score: 0.8463
Epoch 9/10 | F1-score: 0.8363
Epoch 10/10 | F1-score: 0.8567


,client_class_id,client_class_pred
0,0,0
1,1,1
2,2,2
3,2,2
4,1,1
5,1,0
6,3,3
7,1,1
8,3,3
9,0,0


In [7]:
# -----------------------------
# Classification report
# -----------------------------
report = classification_report(
    labels,           # реальные метки на тесте
    preds,            # предсказания модели на тесте
    target_names=["Эконом", "Семейные", "Премиум", "Бизнес"],
    zero_division=0
)
print("Classification Report:")
print(report)

# -----------------------------
# Confusion Matrix
# -----------------------------
cm = confusion_matrix(labels, preds)
print("Confusion Matrix:")
print(cm)


Classification Report:
              precision    recall  f1-score   support

      Эконом       0.62      0.98      0.76        51
    Семейные       0.88      0.86      0.87       228
     Премиум       0.93      0.78      0.85       225
      Бизнес       0.86      0.95      0.90        96

    accuracy                           0.85       600
   macro avg       0.82      0.89      0.84       600
weighted avg       0.87      0.85      0.86       600

Confusion Matrix:
[[ 50   0   1   0]
 [ 12 196  11   9]
 [ 18  25 176   6]
 [  0   3   2  91]]


# Результаты классификации клиентов

## Интерпретация результатов

* **Эконом:** recall = 1.0 → все эконом-заказы правильно классифицированы, но precision ниже (0.63) из-за небольшого числа заказов, часть других классов ошибочно попала сюда.  
* **Семейные:** F1 = 0.85 → большая часть семейных заказов классифицирована верно, но есть смешение с премиум и бизнес.  
* **Премиум:** recall = 0.78 → некоторые премиум-заказы классифицируются как семейные или бизнес, но precision = 0.89 → большинство предсказанных премиум действительно премиум.  
* **Бизнес:** F1 = 0.87 → хорошая классификация, небольшие ошибки в премиум и семейные.  

**Вывод:**  
Модель показывает **достаточно высокую точность** (84%) и уверенно различает классы клиентов. Наибольшие ошибки происходят между **Премиум и Семейными/Групповыми** заказами, что логично, так как характеристики этих заказов могут пересекаться.
